# jsonl파일을 txt파일로 만들기 

- sentencepiece로 토크나이저를 제작하기 위해서는 개행 구분되어진 연속된 txt파일이 필요하다. 그래서 jsonl파일을 txt로 파일로 전환해야 한다. 
- 총 용량이 48GB에 달하는 만큼 멀티프로세싱으로 진행하려 한다. 
- cpu 개수 만큼 chunk로 나누어 임시파일로 저장한 뒤에 멀티프로세싱으로 진행한 뒤에 합친다. 

</br>

- 위처럼 고려했으나 메모리에 한번에 올리지 못하면 사실상 병렬처리를 할수가 없다. 그렇기 때문에 병렬처리를 할 이유가없다. 지금 시스템 메모리가 64GB라 통으로 메모리에 올린뒤 병렬 처리 하는 것은 비효율적이다. 사실 메모리로 읽어 오는 시간이 문제기 때문이다. 그냥 단일 프로세스로 진행하려 한다. 
- 11분 8초 소요

In [1]:
import json
from multiprocessing import Pool, cpu_count
import os

In [1]:
# .jsonl 파일 경로
jsonl_file_path = '/mnt/t7/dnn/llm_practicing/korean_data/aihub_merged.jsonl'
# 최종 출력 파일 경로
output_text_file_path = '/mnt/t7/dnn/llm_practicing/korean_data/aihub_merged.txt'

In [7]:
# .jsonl 파일을 읽고 각 객체의 "target" 값을 추출하여 새 파일에 저장
with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file, \
     open(output_text_file_path, 'w', encoding='utf-8') as output_file:
    for line in jsonl_file:
        # 각 줄을 JSON 객체로 변환
        json_obj = json.loads(line)
        # "target" 필드의 값을 추출하여 새 파일에 쓰기
        output_file.write(json_obj["text"] + '\n')

# 이제 output.txt 파일을 SentencePiece 모델 훈련의 입력으로 사용


In [3]:
import sentencepiece as spm

In [ ]:
prefix = "newtoken-sp-bpe-nsmc-byte-fallback"
vocab_size = 31900-7
spm.SentencePieceTrainer.train(
    f"--input={output_text_file_path} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이 -> 이게 너무 길면 에러발생함
    " --pad_id=0 --pad_piece=<pad>" + # pad (0)
    " --unk_id=1 --unk_piece=<unk>" + # unknown (1)
    " --bos_id=2 --bos_piece=<s>" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=</s>" + # end of sequence (3)
    " --byte_fallback=true" + # add byte_fallback for unk tokens
    " --user_defined_symbols=<sep>,<cls>,<mask>") # 사용자 정의 토큰

: 